https://pythonlib.ru/library-theme9

# Введение

**CatBoost (Categorical Boosting)** - современный алгоритм градиентного бустинга, созданный командой разработчиков Яндекса. Эта библиотека машинного обучения специально оптимизирована для работы с категориальными признаками и предназначена для решения широкого спектра задач: от классификации и регрессии до ранжирования и рекомендательных систем.

**Ключевые преимущества CatBoost:**

*Автоматическая обработка категориальных признаков.*
CatBoost революционизирует подход к работе с категориальными данными. 
Традиционные алгоритмы требуют предварительного преобразования категориальных переменных через one-hot encoding или label encoding. 
CatBoost же использует собственный метод статистической агрегации по истории (CTR - Click-Through Rate), который автоматически обрабатывает категориальные признаки без потери информации.

*Высокая точность и устойчивость к переобучению.*
Алгоритм демонстрирует отличные результаты на различных типах данных благодаря использованию симметричных деревьев и продвинутых методов регуляризации. Встроенные механизмы предотвращения переобучения позволяют получать стабильные результаты даже на небольших выборках.

*Гибкость в вычислительных ресурсах.*
CatBoost поддерживает как CPU-, так и GPU-обучение, что позволяет значительно ускорить процесс обучения на больших объемах данных. Библиотека эффективно работает как с небольшими наборами данных, так и с большими промышленными датасетами.

*Интеграция с популярными инструментами.*
Полная совместимость с экосистемой Python для анализа данных: Pandas, NumPy, Scikit-learn. Это обеспечивает легкую интеграцию в существующие пайплайны машинного обучения.

*Встроенные инструменты анализа.*
CatBoost предоставляет богатые возможности для визуализации процесса обучения, анализа важности признаков и мониторинга качества модели в реальном времени.


**Технические особенности алгоритма:**

*Обработка категориальных признаков.*
Главное отличие CatBoost от других алгоритмов бустинга заключается в способе обработки категориальных переменных. Вместо традиционных методов кодирования, CatBoost использует target-based статистики, вычисляемые на основе исторических данных. Это позволяет сохранить всю информацию о категориальных признаках без увеличения размерности данных.

*Симметричные деревья.*
CatBoost строит симметричные (сбалансированные) деревья, что обеспечивает лучшую обобщающую способность и стабильность предсказаний. Это отличается от подхода XGBoost и LightGBM, которые используют листовые деревья.

*Обработка пропущенных значений.*
Алгоритм автоматически обрабатывает пропущенные значения без необходимости дополнительной предобработки данных. CatBoost рассматривает пропуски как отдельную категорию и эффективно их обрабатывает.

*Поддержка текстовых признаков.*
CatBoost может работать с текстовыми данными, автоматически извлекая признаки из текста и используя их для обучения модели.

**Особенности:**

- CatBoost НЕ принимает NaN в категориальных признаках, но принимает в числовых

In [ ]:
# !pip install catboost
# !pip install catboost[gpu]

In [ ]:
from catboost import CatBoostClassifier, CatBoostRegressor, CatBoostRanker
from catboost import Pool, cv, sum_models

# Подготовка данных 

In [1]:
from catboost.datasets import titanic

titanic_train, titanic_val = titanic()

In [2]:
titanic_train.head(1)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S


In [3]:
titanic_val.head(1)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q


## Работа с категориальными признаками
CatBoost может автоматически определять категориальные признаки, но для лучшего контроля рекомендуется указывать их явно:

In [4]:
import pandas as pd
from catboost import CatBoostClassifier

# Загрузка данных
df = titanic_train

# Определение категориальных признаков
cat_features = ['Sex', 'Embarked', 'Cabin']

for col in cat_features:
    df[col] = df[col].fillna('Missing')

# Подготовка данных
y_feature = 'Survived'
X_drop_features = ['Name', 'Ticket']
X_drop_features_plus_y = X_drop_features + [y_feature]
X = df.drop(X_drop_features_plus_y, axis=1)
y = df[y_feature]

In [ ]:
X.head(1)

## Использование Pool для оптимизации !
Pool - это специальный объект CatBoost для хранения данных, который обеспечивает более эффективную работу с большими датасетами: 

In [ ]:
# from catboost import Pool

# # Создание Pool объекта
# train_pool = Pool(
#     data=X_train,
#     label=y_train,
#     cat_features=cat_features,
#     feature_names=list(X_train.columns)
# )

# eval_pool = Pool(
#     data=X_eval,
#     label=y_eval,
#     cat_features=cat_features,
#     feature_names=list(X_eval.columns)
# )

# Обучение моделей классификации
Базовый пример

In [5]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Создание и обучение модели
model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.01,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    verbose=100,
    custom_loss=['Accuracy'],
)

In [6]:
print(model.is_fitted())
print(model.get_params())

False
{'iterations': 1000, 'learning_rate': 0.01, 'depth': 6, 'loss_function': 'Logloss', 'random_seed': 42, 'verbose': 100, 'custom_loss': ['Accuracy'], 'eval_metric': 'AUC'}


In [7]:
# Обучение с валидацией
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test),
    # early_stopping_rounds=50,
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.8165349	best: 0.8165349 (0)	total: 149ms	remaining: 2m 28s
100:	test: 0.8344532	best: 0.8399868 (90)	total: 909ms	remaining: 8.09s
200:	test: 0.8388669	best: 0.8399868 (90)	total: 1.7s	remaining: 6.77s
300:	test: 0.8412385	best: 0.8416337 (296)	total: 2.53s	remaining: 5.88s
400:	test: 0.8442688	best: 0.8447958 (395)	total: 3.32s	remaining: 4.96s
500:	test: 0.8451910	best: 0.8459816 (475)	total: 4.11s	remaining: 4.1s
600:	test: 0.8483531	best: 0.8490119 (592)	total: 4.87s	remaining: 3.23s
700:	test: 0.8484848	best: 0.8490119 (592)	total: 5.7s	remaining: 2.43s
800:	test: 0.8512516	best: 0.8513834 (789)	total: 6.55s	remaining: 1.63s
900:	test: 0.8499341	best: 0.8520422 (817)	total: 7.41s	remaining: 814ms
999:	test: 0.8501976	best: 0.8520422 (817)	total: 8.29s	remaining: 0us

bestTest = 0.8520421607
bestIteration = 817

Shrink model to first 818 iterations.


In [8]:
# Предсказание
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)

In [9]:
# Оценка качества
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.8045
              precision    recall  f1-score   support

           0       0.82      0.87      0.85       110
           1       0.77      0.70      0.73        69

    accuracy                           0.80       179
   macro avg       0.80      0.78      0.79       179
weighted avg       0.80      0.80      0.80       179



## validation

In [10]:
# Загрузка данных
df_val = titanic_val

for col in cat_features:
    df_val[col] = df_val[col].fillna('Missing')

X_val = df_val.drop(X_drop_features, axis=1)

y_pred_val = model.predict(X_val)

df_val_finish = pd.concat([df_val['PassengerId'], pd.Series(y_pred_val, name='Survived')], axis=1)

df_val_finish = df_val_finish.set_index('PassengerId')

# df_val_finish.to_csv('tit.csv')

# Настройка гиперпараметров

In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV

# Определение сетки параметров
param_grid = {
    # 'iterations': [500, 1000, 1500],
    'learning_rate': [0.01, 0.1],
    'depth': [4, 6],
    'l2_leaf_reg': [3, 5]
}

# Создание модели
model = CatBoostClassifier(
    random_seed=42,
    verbose=0,
    cat_features=cat_features
)

# Поиск по сетке
grid_search = GridSearchCV(
    model, param_grid, 
    cv=3, scoring='roc_auc',
    n_jobs=-1, verbose=1
)

grid_search.fit(X_train, y_train)
print(f"Лучшие параметры: {grid_search.best_params_}")
print(f"Лучший результат: {grid_search.best_score_:.4f}")

In [ ]:
# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Создание и обучение модели
model = CatBoostClassifier(
    # iterations=1000,
    learning_rate=0.01,
    depth=4,
    l2_leaf_reg=3,
    loss_function='Logloss',
    eval_metric='Accuracy',
    random_seed=42,
    verbose=100
)

# Обучение с валидацией
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test),
    # early_stopping_rounds=50,
    plot=True
)

In [ ]:
# Загрузка данных
df_val = titanic_val

for col in cat_features:
    df_val[col] = df_val[col].fillna('Missing')

X_val = df_val.drop(X_drop_features, axis=1)

y_pred_val = model.predict(X_val)

df_val_finish = pd.concat([df_val['PassengerId'], pd.Series(y_pred_val, name='Survived')], axis=1)

df_val_finish = df_val_finish.set_index('PassengerId')

df_val_finish.to_csv('tit.csv')

# Обучение моделей регрессии

In [11]:
from catboost.datasets import monotonic2
monotonic2_train, monotonic2_test = monotonic2()

In [12]:
monotonic2_train.head(3)

,Target,MonotonicNeg0,MonotonicPos0,MonotonicPos1,MonotonicNeg1
0,0.000000,0.810332,0.526106,0.005630,0.007460
1,0.166667,NaN,NaN,0.002114,0.004091
2,0.333333,NaN,NaN,0.008012,0.014145


In [13]:
monotonic2_train.describe()

,Target,MonotonicNeg0,MonotonicPos0,MonotonicPos1,MonotonicNeg1
count,3237.000000,1837.000000,1837.000000,3237.000000,3237.000000
mean,0.476882,0.741484,0.746498,0.048551,0.050682
std,0.373377,0.182314,0.183029,0.146101,0.148566
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.166667,0.636268,0.649659,0.003947,0.004115
50%,0.500000,0.780167,0.786654,0.008819,0.008935
75%,0.833333,0.887218,0.890239,0.020626,0.021137
max,1.000000,0.998125,0.999058,1.000000,1.000000


In [14]:
X_train = monotonic2_train.drop(['Target'], axis=1)
X_test = monotonic2_test.drop(['Target'], axis=1)

In [15]:
y_train = monotonic2_train['Target']
y_test = monotonic2_test['Target']

In [16]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error

cat_features = []

# Создание модели регрессии
regressor = CatBoostRegressor(
    iterations=1000,
    learning_rate=0.05,
    depth=8,
    loss_function='RMSE',
    eval_metric='MAE',
    random_seed=42,
    verbose=100
)

# Обучение
regressor.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_test, y_test),
    early_stopping_rounds=100
)

0:	learn: 0.3277448	test: 0.3264004	best: 0.3264004 (0)	total: 2.05ms	remaining: 2.04s
100:	learn: 0.2174053	test: 0.2375127	best: 0.2375127 (100)	total: 167ms	remaining: 1.49s
200:	learn: 0.1986968	test: 0.2297192	best: 0.2297192 (200)	total: 336ms	remaining: 1.33s
300:	learn: 0.1872919	test: 0.2283090	best: 0.2282075 (276)	total: 507ms	remaining: 1.18s
400:	learn: 0.1781759	test: 0.2274509	best: 0.2274509 (400)	total: 674ms	remaining: 1.01s
500:	learn: 0.1703934	test: 0.2272069	best: 0.2270762 (453)	total: 838ms	remaining: 834ms
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2270761868
bestIteration = 453

Shrink model to first 454 iterations.


In [17]:
# Предсказание
y_pred = regressor.predict(X_test)

# Оценка
mae = mean_absolute_error(y_test, y_pred)
print(f"MAE: {mae:.4f}")
r2_score_ = r2_score(y_test, y_pred)
print(f"r2_score_: {r2_score_:.4f}")
root_mean_squared_error_ = root_mean_squared_error(y_test, y_pred)
print(f"root_mean_squared_error_: {root_mean_squared_error_:.4f}")

MAE: 0.2271
r2_score_: 0.4306
root_mean_squared_error_: 0.2800


# Кросс-валидация
Автоматически подбирает модель (классикация / регрессия) по параметру loss_function.

Служит для быстрого подбора гиперпараметров на кросс-валидации

In [18]:
from catboost import cv
from catboost import Pool
from sklearn.model_selection import cross_val_score

# Использование встроенной кросс-валидации CatBoost
cv_results = cv(
    pool=Pool(X_train, y_train, cat_features=[]),
    params={
        'iterations': 1000,
        'learning_rate': 0.1,
        'depth': 6,
        'loss_function': 'RMSE'
    },
    fold_count=5,
    shuffle=True,
    stratified=True, # для равномерного распределения классов
    seed=42,
    verbose=100,
    plot=True
)

# cv_results
print(f"Среднее значение RMSE: {cv_results['test-RMSE-mean'].iloc[-1]:.4f}")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]
0:	learn: 0.5652674	test: 0.5650780	best: 0.5650780 (0)	total: 1.06ms	remaining: 1.06s
100:	learn: 0.2547405	test: 0.2957468	best: 0.2955963 (99)	total: 75ms	remaining: 668ms
200:	learn: 0.2347382	test: 0.2958317	best: 0.2948355 (124)	total: 147ms	remaining: 585ms
300:	learn: 0.2206240	test: 0.2983331	best: 0.2948355 (124)	total: 221ms	remaining: 513ms
400:	learn: 0.2084144	test: 0.2989742	best: 0.2948355 (124)	total: 294ms	remaining: 439ms
500:	learn: 0.1982599	test: 0.3005649	best: 0.2948355 (124)	total: 367ms	remaining: 365ms
600:	learn: 0.1888631	test: 0.3025406	best: 0.2948355 (124)	total: 441ms	remaining: 293ms
700:	learn: 0.1816191	test: 0.3033050	best: 0.2948355 (124)	total: 515ms	remaining: 220ms
800:	learn: 0.1747222	test: 0.3043917	best: 0.2948355 (124)	total: 588ms	remaining: 146ms
900:	learn: 0.1685820	test: 0.3055750	best: 0.2948355 (124)	total: 666ms	remaining: 73.2ms
999:	learn: 0.1630008	test: 0.3068916	best: 0.2948355 (124)	total: 748ms	remainin

In [19]:
cv_results

,iterations,test-RMSE-mean,test-RMSE-std,train-RMSE-mean,train-RMSE-std
0,0,0.565682,0.000607,0.565535,0.000402
1,1,0.530137,0.001082,0.529721,0.000453
2,2,0.498765,0.001871,0.498168,0.000483
3,3,0.472475,0.001912,0.471595,0.000470
4,4,0.449307,0.001773,0.448169,0.000274
...,...,...,...,...,...
995,995,0.289477,0.012240,0.163740,0.002441
996,996,0.289533,0.012236,0.163695,0.002450
997,997,0.289543,0.012240,0.163653,0.002413
998,998,0.289560,0.012253,0.163599,0.002428
